# MMI prediction

## 6. Testing
- Load test data and models
- Test models
- Get permutation feature importance values
- Evaluate performance of individual risk factors

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

from pickle import load

from sklearn.utils import resample
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.inspection import permutation_importance

from tensorflow.keras.models import load_model

In [2]:
# load testing data

X_test = pd.read_pickle('X_test_processed.pkl')
y_test = pd.read_pickle('y_test.pkl')
print('there are {} testing samples'.format(X_test.shape[0]))

there are 77 testing samples


In [5]:
y_test.value_counts()

N    66
Y    11
Name: malig_infarct, dtype: int64

In [6]:
le = LabelEncoder()
y_test = le.fit_transform(y_test)

In [7]:
os.listdir('models')

['final_rf.sav',
 'final_logreg.sav',
 'final_mlp.h5',
 'final_mlp',
 'final_svm.sav',
 '.ipynb_checkpoints']

## Load models

In [8]:
# helper function to load models

def load_models():
    final_models = []

    for model in os.listdir('models'):
        if model.endswith('.sav'):
            test_model = load(open(os.path.join('models', model), 'rb'))
            final_models.append((model.split('.')[0], test_model))
        if model == 'final_mlp':
            test_model = load_model(os.path.join('models', model))
            final_models.append((model, test_model))
    return final_models

In [9]:
# load models

final_models = load_models()

In [10]:
final_models

[('final_rf',
  RandomForestClassifier(min_samples_leaf=2, min_samples_split=5,
                         n_estimators=140)),
 ('final_logreg', LogisticRegression(C=0.1, max_iter=1000)),
 ('final_mlp', <keras.engine.functional.Functional at 0x7f9443e57100>),
 ('final_svm', SVC(C=1, gamma=0.0001, probability=True))]

## Test models

In [82]:
# helper function to evaluate models multiple times with bootstrapping and save results

def eval_model(final_models, n_iterations, fname):

    # create empty results dicts for each metric
    results_auc = {'final_rf': [], 'final_logreg': [], 'final_mlp': [], 'final_svm': []}
    results_prec = {'final_rf': [], 'final_logreg': [], 'final_mlp': [], 'final_svm': []}
    results_recall = {'final_rf': [], 'final_logreg': [], 'final_mlp': [], 'final_svm': []}
    results_f1 = {'final_rf': [], 'final_logreg': [], 'final_mlp': [], 'final_svm': []}
    results_acc = {'final_rf': [], 'final_logreg': [], 'final_mlp': [], 'final_svm': []}
    
    # bootstrap resample
    for i in range(n_iterations):
        
        if i % 50 == 0:
            print('evaluating iteration no. {}...'.format(i))
        
        X_test_resampled, y_true = resample(X_test, y_test)
        
        # get scores for each model
        for model_name, Model in final_models:
            
            # keras model
            if model_name == 'final_mlp':
                auc = roc_auc_score(y_true, Model.predict(X_test_resampled))
                results_auc[model_name].append(auc)
                
                y_pred = (Model.predict(X_test_resampled) > 0.5).astype(int)
                
                prec = precision_score(y_true, y_pred, zero_division = 0)
                results_prec[model_name].append(prec)
                recall = recall_score(y_true, y_pred)
                results_recall[model_name].append(recall)
                f1 = f1_score(y_true, y_pred)
                results_f1[model_name].append(f1)
                acc = accuracy_score(y_true, y_pred)
                results_acc[model_name].append(acc) 

            # sklearn models
            else:
                auc = roc_auc_score(y_true, Model.predict_proba(X_test_resampled)[:, 1])
                results_auc[model_name].append(auc)

                y_pred = Model.predict(X_test_resampled)

                prec = precision_score(y_true, y_pred, zero_division = 0)
                results_prec[model_name].append(prec)
                recall = recall_score(y_true, y_pred)
                results_recall[model_name].append(recall)
                f1 = f1_score(y_true, y_pred)
                results_f1[model_name].append(f1)
                acc = accuracy_score(y_true, y_pred)
                results_acc[model_name].append(acc)    
    
    
    # create data frames for each metric
    pd.DataFrame(results_auc).to_pickle('test_results/df_auc.pkl')
    pd.DataFrame(results_prec).to_pickle('test_results/df_prec.pkl')
    pd.DataFrame(results_recall).to_pickle('test_results/df_recall.pkl')
    pd.DataFrame(results_f1).to_pickle('test_results/df_f1.pkl')
    pd.DataFrame(results_acc).to_pickle('test_results/df_acc.pkl')
    
    # create summary txt file with results
    combined_results = [('auc', results_auc), ('prec', results_prec), ('recall', results_recall), 
                        ('f1', results_f1), ('acc', results_acc)]
    result_file = open(fname, 'a')
    for name, result in combined_results:
        result_file.write('\n\n' + 'results for: ' + name + '\n')
        for model in result:
            result_file.write(model + ': ' + str(np.median(result[model])))
    result_file.close()

In [81]:
# test models

iter = 1000

fname = 'test_results/test_summary' + '_' + str(datetime.now().year) + '_' + str(datetime.now().month) \
    + '_' + str(datetime.now().day) + '_' + str(datetime.now().hour) + '_' + \
    str(datetime.now().minute) + '.txt'

eval_model(final_models = final_models, n_iterations = iter, fname = fname)

evaluating iteration no. 0...
evaluating iteration no. 50...
evaluating iteration no. 100...
evaluating iteration no. 150...
evaluating iteration no. 200...
evaluating iteration no. 250...
evaluating iteration no. 300...
evaluating iteration no. 350...
evaluating iteration no. 400...
evaluating iteration no. 450...
evaluating iteration no. 500...
evaluating iteration no. 550...
evaluating iteration no. 600...
evaluating iteration no. 650...
evaluating iteration no. 700...
evaluating iteration no. 750...
evaluating iteration no. 800...
evaluating iteration no. 850...
evaluating iteration no. 900...
evaluating iteration no. 950...


## Feature importances

In [85]:
# get feature importances for each sklearn model

for model_name, Model in final_models:
    if model_name != 'final_mlp':
        feat_imp = permutation_importance(Model, X_test, y_test, n_repeats = 10)
        feat_imp_df = pd.DataFrame(feat_imp.importances, index = X_test.columns.tolist())
        feat_imp_df.to_pickle('test_results/feat_imp_{}.pkl'.format(model_name))

## Individual risk factors

In [2]:
# load unprocessed testing data

X_test = pd.read_pickle('X_test.pkl')
y_test = pd.read_pickle('y_test.pkl')
le = LabelEncoder()
y_test = le.fit_transform(y_test)

In [3]:
def age_prediction(X_test):
    
    y = np.where(X_test['age'] < 55, 1, 0)
    
    return y

In [4]:
def diabetes_prediction(X_test):
    
    y = np.where(X_test['dm'] == 'Y', 1, 0)
    
    return y

In [5]:
def nih_prediction(X_test, threshold):

    y = np.where(X_test['nih_admit'] > threshold, 1, 0)
    
    return y

In [6]:
def revasc_prediction(X_test):
    
    y = np.where(X_test['time_to_reperf'] > 7, 1, 0)
    
    return y

In [7]:
def aspects_prediction(X_test):
    
    y = np.where(X_test['aspects'] < 8, 1, 0)
    
    return y

In [8]:
def mbe_score(X_test):
    
    aspects = np.where(X_test['aspects'] < 8, 1, 0)
    
    conditions = [X_test['nih_admit'] > 17, (X_test['nih_admit'] < 18) & (X_test['nih_admit'] > 8)]
    choices = [2, 1]
    nihss = np.select(conditions, choices, 0)
    
    collateral = np.where(X_test['coll_score'] < 2, 2, 0)
    revasc = np.where((X_test['tici'] == '2B') | (X_test['tici'] == '3'), 0, 1)
    
    components = np.array([aspects, nihss, collateral, revasc])
    mbe = components.sum(axis = 0)
    
    y = np.where(mbe > 3, 1, 0)
    
    return y

In [9]:
# helper function to evaluate risk factors

def eval_scores(n_iterations):
    
    # create empty results dicts for each metric
    results_auc = {'age': [], 'diabetes': [], 'nih_10': [], 'nih_15': [], 'nih_20': [], 'revasc': [], 'aspects': [], 
               'mbe': []}
    results_acc = {'age': [], 'diabetes': [], 'nih_10': [], 'nih_15': [], 'nih_20': [], 'revasc': [], 'aspects': [], 
               'mbe': []}
    
    # bootstrap resample
    for i in range(n_iterations):
        
        if i % 50 == 0:
            print('evaluating iteration no. {}...'.format(i))
        
        X_test_resampled, y_true = resample(X_test, y_test)
        
        # get scores for each model
        y_pred = age_prediction(X_test = X_test_resampled)
        auc = roc_auc_score(y_true, y_pred)
        results_auc['age'].append(auc)
        acc = accuracy_score(y_true, y_pred)
        results_acc['age'].append(acc)
        
        y_pred = diabetes_prediction(X_test = X_test_resampled)
        auc = roc_auc_score(y_true, y_pred)
        results_auc['diabetes'].append(auc)
        acc = accuracy_score(y_true, y_pred)
        results_acc['diabetes'].append(acc)
        
        y_pred = nih_prediction(X_test = X_test_resampled, threshold = 10)
        auc = roc_auc_score(y_true, y_pred)
        results_auc['nih_10'].append(auc)
        acc = accuracy_score(y_true, y_pred)
        results_acc['nih_10'].append(acc)
        
        y_pred = nih_prediction(X_test = X_test_resampled, threshold = 15)
        auc = roc_auc_score(y_true, y_pred)
        results_auc['nih_15'].append(auc)
        acc = accuracy_score(y_true, y_pred)
        results_acc['nih_15'].append(acc)
        
        y_pred = nih_prediction(X_test = X_test_resampled, threshold = 20)
        auc = roc_auc_score(y_true, y_pred)
        results_auc['nih_20'].append(auc)
        acc = accuracy_score(y_true, y_pred)
        results_acc['nih_20'].append(acc)
        
        y_pred = revasc_prediction(X_test = X_test_resampled)
        auc = roc_auc_score(y_true, y_pred)
        results_auc['revasc'].append(auc)
        acc = accuracy_score(y_true, y_pred)
        results_acc['revasc'].append(acc)
        
        y_pred = aspects_prediction(X_test = X_test_resampled)
        auc = roc_auc_score(y_true, y_pred)
        results_auc['aspects'].append(auc)
        acc = accuracy_score(y_true, y_pred)
        results_acc['aspects'].append(acc)
        
        y_pred = mbe_score(X_test = X_test_resampled)
        auc = roc_auc_score(y_true, y_pred)
        results_auc['mbe'].append(auc)
        acc = accuracy_score(y_true, y_pred)
        results_acc['mbe'].append(acc)
        
    results_auc = pd.DataFrame(results_auc)
    results_acc = pd.DataFrame(results_acc)
    results_auc.to_pickle('test_results/risk_factor_scores_auc.pkl')
    results_acc.to_pickle('test_results/risk_factor_scores_acc.pkl')
    
    return results_auc, results_acc

In [10]:
iter = 1000

results_auc, results_acc = eval_scores(n_iterations = iter)

evaluating iteration no. 0...
evaluating iteration no. 50...
evaluating iteration no. 100...
evaluating iteration no. 150...
evaluating iteration no. 200...
evaluating iteration no. 250...
evaluating iteration no. 300...
evaluating iteration no. 350...
evaluating iteration no. 400...
evaluating iteration no. 450...
evaluating iteration no. 500...
evaluating iteration no. 550...
evaluating iteration no. 600...
evaluating iteration no. 650...
evaluating iteration no. 700...
evaluating iteration no. 750...
evaluating iteration no. 800...
evaluating iteration no. 850...
evaluating iteration no. 900...
evaluating iteration no. 950...


In [11]:
results_auc.describe()

,age,diabetes,nih_10,nih_15,nih_20,revasc,aspects,mbe
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.498242,0.556858,0.549609,0.595851,0.476577,0.526335,0.548767,0.521589
std,0.064799,0.078616,0.065141,0.081225,0.064881,0.083879,0.063448,0.047246
min,0.345070,0.320896,0.345833,0.328431,0.326087,0.273881,0.392308,0.423077
25%,0.452524,0.504681,0.505878,0.539566,0.429087,0.469697,0.503205,0.484375
50%,0.494158,0.554125,0.553128,0.594363,0.472133,0.527115,0.550000,0.518590
75%,0.540320,0.606370,0.592952,0.652364,0.518540,0.578358,0.584978,0.548007
max,0.778595,0.815359,0.719697,0.833333,0.749183,0.818841,0.790761,0.741013


In [12]:
results_acc.describe()

,age,diabetes,nih_10,nih_15,nih_20,revasc,aspects,mbe
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.723727,0.700260,0.362273,0.569662,0.686636,0.574935,0.804779,0.829675
std,0.050072,0.052429,0.054085,0.058326,0.050455,0.055771,0.045620,0.041874
min,0.558442,0.532468,0.207792,0.324675,0.506494,0.389610,0.636364,0.688312
25%,0.688312,0.662338,0.324675,0.532468,0.649351,0.532468,0.779221,0.805195
50%,0.727273,0.701299,0.363636,0.571429,0.688312,0.571429,0.805195,0.831169
75%,0.753247,0.740260,0.402597,0.610390,0.727273,0.610390,0.831169,0.857143
max,0.870130,0.857143,0.558442,0.766234,0.831169,0.753247,0.935065,0.961039
